In [1]:
import xarray as xr
import matplotlib.pyplot as plt
import warnings 
warnings.filterwarnings("ignore")

from tqdm import tqdm

import cartopy, cartopy.crs as ccrs, cartopy.feature as cfeature, cartopy.mpl.ticker as cticker
from cartopy.feature import NaturalEarthFeature
from cartopy.util import add_cyclic_point

from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)
import numpy as np

from math import sin, cos, sqrt, atan2, radians 

In [ ]:
# Approximating the radius of earth in km
R = 6373.0

def get_distance(lat1, lon1, lat2, lon2):
    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    a = sin((lat2 - lat1) / 2)**2 + cos(lat1) * cos(lat2) * sin((lon2 - lon1) / 2)**2
    
    return R * (2 * atan2(sqrt(a), sqrt(1 - a)))

In [ ]:
# Calculating the minimum distance between each transect and stations, 
# and assigning the index of the station with the minimum distance to each transect, 
# as long as the minimum distance is less than 100. The assignments are stored in a dictionary called assignment

assignment = {}
for i, transect in enumerate(coastline_bbox.transect):
    x = [] # beinhalted dann die distanzen zwischen coastline und transact
    for i, station in enumerate(sealevel_bbox.station_name): 
        x.append(
            get_distance(
                station.lat, # lat1
                station.lon, # lon1
                transect["lat"], # lat2
                transect["lon"]  # lon2
            )
        )
    
    min_diff = min(x)
    if min_diff < 100:
        assignment[str(i)] = x.index(min_diff)

In [1]:
# Assigning specific station indices to corresponding transects in the assignment dictionary. 
# Each key-value pair in the dictionary represents the transect index and the index of the station 
# that is closest to that transect

assignment = {
# station, transect
    "1": 7, # transect 1 ist an station 7 am nähesten dran
    "2": 9, # transect 2 ist an station 9 am nähesten dran
    "199": 7 # transect 199 ist an station 7 am nähesten dran
}

In [ ]:
# Initialize empty lists for x and y coordinates
x, y = [], []

# Extract the indices of transects and stations from the assignment dictionary
transects_idxs = [int(k) for k in assignment.keys()]
station_idxs = assignment.values()

# Loop through each transect and station index pair
for transect_idx, station_idx in zip(transects_idxs, station_idxs):
    # Append x-coordinate (coastline activeZone2Land value) to the x list
    x.append(coastline_bbox[transect_idx]["activeZone2Land"].values.ravel()[0])
    
    # Append y-coordinate (difference in sealevel height) to the y list
    y.append(sealevel_bbox[station_idx]["hr"][0] - sealevel_bbox[station_idx]["hr"][-1])
    # Alternatively, you can calculate the mean of sealevel height differences across time using:
    # y.append(sealevel_bbox[station_idx]["hr"].mean("time"))
